# Toy model
This is a toy model of a diagnosis system based on the MSDX data and 5MCC document.This demo is interactive and adaptive. Given the input from the patients or physicians, a question is asked based on their answer, and a diagnosis is made if certain crtiria is met. 

## 1. load data in the back-end

In [212]:
import pandas as pd
dist_f = pd.read_csv('../MarketScan/nfemale.csv', dtype=str, index_col=0)
dist_m = pd.read_csv('../MarketScan/nmale.csv', dtype=str, index_col=0)
sym2code = pd.read_csv('../ClinicalConsult/Sym2ICD.csv', index_col=0)
code2sym = pd.read_csv('../ClinicalConsult/ICD2Sym.csv', index_col=0)

# Strings to lists
def s2list(s):
    return s.strip("['").strip("']").split("', '")
sym2code['Codes'] = sym2code['Codes'].apply(s2list)
code2sym['Symptoms'] = code2sym['Symptoms'].apply(s2list)

# Symptoms to ICD codes
s2c = {}
for i in sym2code.index:
    s2c[sym2code.loc[i]['Symptoms']] = sym2code.loc[i]['Codes']
    
c2s = {}
for i in code2sym.index:
    c2s[code2sym.loc[i]['Codes']] = code2sym.loc[i]['Symptoms']

## 2. The main diagnosis process

In [213]:
def diagnosis():
    # initial input part
    gendermap = {'0':'Female', '1': 'Male'}
    print('Please type in the gender for the patient. 0 for female and 1 for male')
    g = input()
    gender = gendermap[g]
    print('Please type in the age for the patient in years.')
    age = int(input())
    print('What symptom do you have?')
    sym = input()
    
    Symptoms = set()
    
    if gender == 'Male':
        Dis_list = dist_m[str(age)]
    elif gender == 'Female':
        Dis_list = dist_f[str(age)]
    else:
        print('Sorry your gender is not recognized.')
        return -1
    Dis_list=Dis_list.apply(float)
    Dis_list.dropna(inplace=True)
    for i in Dis_list.index:
        if Dis_list[i]<10:
            Dis_list.drop(i, inplace=True)
    
    
    Diseases = {}
    for s in s2c.keys():
        if sym in str(s):
            Symptoms.add(s)
            for dis in s2c[s]:
                if dis in Dis_list:
                    Diseases[dis] = Dis_list[dis]
    
    print("Initial guess for the diagnosis includes:")
    print(Diseases.keys())
    
    # Start diagnosis
    while True:    
        total = sum(Diseases.values())
        for i in Diseases.keys():
            Diseases[i] /= total
        Dis_sorted = sorted(Diseases.items(), key = lambda item: item[1], reverse=True)
        
        slist = list(set(c2s[Dis_sorted[0][0]]) - set(Symptoms))
        
        if max(Diseases.values()) > 0.8 or len(Diseases) == 1 or len(slist)==0:
            print('Diagnosis done. The possible diseases are:')
            print(Dis_sorted)
            return 0
        
        print('Do you have the following symptom: (Y for Yes and N for No)')
        print(slist[0])
        answer = input()
        while answer != 'Y' and answer != 'N':
            answer = input()
        
        if answer == 'Y':
            Symptoms.add(slist[0])
            Diseases[Dis_sorted[0][0]] *= 2
        else:
            Diseases.pop(Dis_sorted[0][0])
        

In [211]:
diagnosis()

Please type in the gender for the patient. 0 for female and 1 for male
1
Please type in the age for the patient in years.
10
What symptom do you have?
cough
Initial guess for the diagnosis includes:
dict_keys(['034.0', '993.2', '464.10', '464.20', '530.81', '495.9', '117.9', '494.0', '033.9', '162.9', '560.0', '491.8', '989.5', '995.0', '493.00', '788.30', '115.90', '492.8', '464.30', '300.7', '693.1', '483.0', '428.0', '078.5', '287.0', '507.0', '394.0', '088.82', '446.1', '520.7'])
Do you have the following symptom: (Y for Yes and N for No)
Sore throat
Y
Do you have the following symptom: (Y for Yes and N for No)
Tonsillar exudates
Y
Diagnosis done. The possible diseases are:
[('034.0', 0.87266149194086162), ('493.00', 0.060345935512343921), ('530.81', 0.026894509795363113), ('788.30', 0.010994473664428489), ('693.1', 0.010519194255908861), ('989.5', 0.0069133754780074143), ('483.0', 0.0030796165756118614), ('995.0', 0.0017628986224171835), ('464.10', 0.0014840357041531174), ('464.20

0

In [215]:
diagnosis()

Please type in the gender for the patient. 0 for female and 1 for male
1
Please type in the age for the patient in years.
24
What symptom do you have?
cough
Initial guess for the diagnosis includes:
dict_keys(['034.0', '993.2', '464.10', '464.20', '530.81', '495.9', '117.9', '494.0', '055.9', '033.9', '162.9', '560.0', '491.8', '989.5', '002.0', '482.83', '995.0', '493.00', '788.30', '115.90', '492.8', '464.30', '300.7', '693.1', '483.0', '428.0', '078.5', '287.0', '507.0', '394.0', '088.82', '446.1', '078.88', '446.5'])
Do you have the following symptom: (Y for Yes and N for No)
Bronchospasm (asthma) (15–20%)
Y
Do you have the following symptom: (Y for Yes and N for No)
Loss of dental enamel
Y
Do you have the following symptom: (Y for Yes and N for No)
Regurgitation of undigested food, sour or acid taste in mouth
Y
Diagnosis done. The possible diseases are:
[('530.81', 0.87198118549563719), ('034.0', 0.065995428637560388), ('493.00', 0.029304665186967064), ('989.5', 0.0074108976981991

0